<a href="https://colab.research.google.com/github/hoodini/PhotoMaker-Colab/blob/main/PhotoMaker_(Anime_Module)_modified_by_Yuval_Avidani_hackit_co_il.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PhotoMaker modified to run on Colab by Yuval Avidani @hackit.co.il  https://linktree.com/hackit.co.il   
# The modification was not an easy job. Please appreciate the hard work by buying me a beer:
# https://www.buymeacoffee.com/hackit.co.il

## How to use:
**NOTE: Make sure you have GPU active! This can be seen under Runtime -> Change Runtime Type -> that you run on GPU (T4 / T4 with High-Ram / V100 / A100). THE NOTEBOOK WILL MOST LIKELY TO CRASH IF NOT USING AT LEAST V100! **  
  
1. Run STEP 1 (Installation)  
2. Create a folder inside the 'examples' folder and upload as many images you like. The image should show the face in most of its area!  
Then, specify the complete folder path by clicking on the folder -> copy path -> and pasting it as the value of the 'input_folder_name' variable (e.g: input_folder_name = '/content/PhotoMaker/examples/newton_man')  
3. STEP 3 - Type in your prompt and click run. That's it!  

In [13]:
!git clone https://github.com/TencentARC/PhotoMaker.git
%cd PhotoMaker/
!pip install torch==2.0.1 torchvision==0.15.2 diffusers==0.25.0 transformers==4.36.2 huggingface-hub==0.20.2 spaces==0.19.4 numpy accelerate safetensors omegaconf peft gradio

import torch
print(torch.__version__)
print(torch.version.cuda)

# Clear CUDA cache
torch.cuda.empty_cache()

import torch
import numpy as np
import random
import os
from PIL import Image
from diffusers.utils import load_image
from diffusers import EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from diffusers import StableDiffusionPipeline
from photomaker.pipeline import PhotoMakerStableDiffusionXLPipeline

# gloal variable and function
def image_grid(imgs, rows, cols, size_after_resize):
    # Calculate the actual number of rows needed
    actual_rows = len(imgs) // cols
    if len(imgs) % cols != 0:
        actual_rows += 1

    w, h = size_after_resize, size_after_resize
    grid = Image.new('RGB', size=(cols*w, actual_rows*h))

    for i, img in enumerate(imgs):
        img = img.resize((w,h))
        grid.paste(img, box=(i%cols*w, i//cols*h))

    return grid

# base_model_path = 'SG161222/RealVisXL_V3.0'

# Download the PhotoMaker model
photomaker_path = hf_hub_download(
    repo_id="TencentARC/PhotoMaker",
    filename="photomaker-v1.bin",
    repo_type="model"
)

base_model_path = 'nitrosocke/Ghibli-Diffusion'
lora_path = './civitai_models/xl_more_art-full.safetensors'

device = "cuda"
save_path = "./outputs"

# Load base model
pipe = PhotoMakerStableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    use_safetensors=True
).to(device)

# Load PhotoMaker checkpoint
pipe.load_photomaker_adapter(
    os.path.dirname(photomaker_path),
    subfolder="",
    weight_name=os.path.basename(photomaker_path),
    trigger_word="img"
)


pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.fuse_lora()

# Load base model
pipe = PhotoMakerStableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    variant="fp16"
).to(device)

# Load PhotoMaker checkpoint
pipe.load_photomaker_adapter(
    os.path.dirname(photomaker_path),
    subfolder="",
    weight_name=os.path.basename(photomaker_path),
    trigger_word="img"
)

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
print("Loading lora...")
pipe.load_lora_weights(os.path.dirname(lora_path), weight_name=os.path.basename(lora_path), adapter_name="xl_more_art-full")
pipe.set_adapters(["photomaker", "xl_more_art-full"], adapter_weights=[1.0, 0.5])
pipe.fuse_lora()

Cloning into 'PhotoMaker'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 162 (delta 59), reused 48 (delta 37), pack-reused 83
Receiving objects: 100% (162/162), 9.30 MiB | 28.50 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker/PhotoMaker
2.0.1+cu117
11.7


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# STEP 2 - create a folder, upload your images, specify the path

In [ ]:
# define and show the input ID images
input_folder_name = '/content/PhotoMaker/examples/123'
image_basename_list = os.listdir(input_folder_name)
image_path_list = sorted([os.path.join(input_folder_name, basename) for basename in image_basename_list])

input_id_images = []
for image_path in image_path_list:
    input_id_images.append(load_image(image_path))

input_grid = image_grid(input_id_images, 1, 4, size_after_resize=224)
print("Input ID images:")
input_grid

# STEP 3 - Generate AI Images

In [ ]:
## Note that the trigger word `img` must follow the class word for personalization
prompt = "ghibli style girl img riding dragon over a whimsical castle, 3d CGI, art by Pixar, half-body, screenshot from animation"
negative_prompt = "realistic, photo-realistic, bad quality, bad anatomy, worst quality, low quality, lowres, extra fingers, blur, blurry, ugly, wrong proportions, watermark, image artifacts, bad eyes, bad hands, bad arms"
generator = torch.Generator(device=device).manual_seed(42)
num_images_per_prompt=1

## Parameter setting
num_steps = 50
style_strength_ratio = 20
start_merge_step = int(float(style_strength_ratio) / 100 * num_steps)
if start_merge_step > 30:
    start_merge_step = 30

# Generate images
images = pipe(
    prompt=prompt,
    input_id_images=input_id_images,
    negative_prompt=negative_prompt,
    num_images_per_prompt=num_images_per_prompt,
    num_inference_steps=num_steps,
    start_merge_step=start_merge_step,
    generator=generator,
).images

# Show and save the results
## Downsample for visualization
grid = image_grid(images, 1, 4, size_after_resize=512)

os.makedirs(save_path, exist_ok=True)
for idx, image in enumerate(images):
    image.save(os.path.join(save_path, f"photomaker_{idx:02d}.png"))

print("Results:")
grid